In [30]:
#importing required packages
import sys
import rdkit
import numpy as np
import pandas as pd
import xgboost as xgb
from rdkit import Chem
from sklearn import svm
from rdkit.Chem import QED
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import metrics
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Lipinski
from xgboost import XGBClassifier
from rdkit.Chem import Descriptors
from sklearn.metrics import f1_score
from sklearn.datasets import load_iris
from rdkit.Chem import rdMolDescriptors
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from rdkit.Chem import Descriptors, Lipinski
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from rdkit.DataStructs import ConvertToNumpyArray
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score

In [31]:
#reading the train csv file
train = pd.read_csv('train_II.csv')
train.shape

#reading the test csv file
test = pd.read_csv('test_II.csv')
test.shape

#Splitting the train data 
train[['Chemicals','AssayID']] = train['Id'].str.split(';',expand=True)

#Splitting the train data 
test[['Chemicals','AssayID']] = test['x'].str.split(';',expand=True)

In [32]:
#smiles - molecules 
#null - discard
#code to generate Molecules from SMILES:

# Read the dataset file containing SMILES strings
def generating_Molecules(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol

# Convert the SMILES strings into molecule objects
train['Feature_1'] = train['Chemicals'].apply(generating_Molecules)
test['Feature_1'] = test['Chemicals'].apply(generating_Molecules)

#code to remove null molecules:
train = train[train['Feature_1'].notnull()]
test = test[test['Feature_1'].notnull()]


[20:15:54] Explicit valence for atom # 1 Si, 8, is greater than permitted
[20:15:57] Explicit valence for atom # 1 Si, 8, is greater than permitted
[20:16:00] Explicit valence for atom # 1 Si, 8, is greater than permitted
[20:16:01] Explicit valence for atom # 1 Si, 8, is greater than permitted
[20:16:03] Explicit valence for atom # 1 Si, 8, is greater than permitted
[20:16:05] Explicit valence for atom # 1 Si, 8, is greater than permitted


In [33]:
#The Feature's column in the train and test datasets contains molecule objects generated from SMILES strings using RDKit library.
#The if-else statement in the list comprehension checks if the molecule is not null before calculating its molweight. 
#If the molecule is null, the corresponding value in the 'Feature_2' column is set to None.

#Extract molecular descriptors
#Feature_2 Molecular weight
train['Feature_2'] = [Descriptors.MolWt(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_2'] = [Descriptors.MolWt(mol) if mol is not None else None for mol in test['Feature_1']]

#Feature_3 NumHeavyAtoms
train['Feature_3'] = [mol.GetNumHeavyAtoms() if mol is not None else None for mol in train['Feature_1']]
test['Feature_3'] = [mol.GetNumHeavyAtoms() if mol is not None else None for mol in test['Feature_1']]

#rotatable bounds
train['Feature_4'] = [Chem.rdMolDescriptors.CalcNumRotatableBonds(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_4'] = [Chem.rdMolDescriptors.CalcNumRotatableBonds(mol) if mol is not None else None for mol in test['Feature_1']]

#MolMR (Molecular Multiple Regression)
train['Feature_5'] = [Descriptors.MolMR(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_5'] = [Descriptors.MolMR(mol) if mol is not None else None for mol in test['Feature_1']]

# Calculate the number of valence electrons
train['Feature_6'] = [Descriptors.NumValenceElectrons(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_6'] = [Descriptors.NumValenceElectrons(mol) if mol is not None else None for mol in test['Feature_1']]

#qed: quantitative estimate of drug-likeness (qed) is a descriptor
train['Feature_8'] = [QED.qed(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_8'] = [QED.qed(mol) if mol is not None else None for mol in test['Feature_1']]

#TPSA stands for Topological Polar Surface Area
train['Feature_9'] = [Descriptors.TPSA(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_9'] = [Descriptors.TPSA(mol) if mol is not None else None for mol in test['Feature_1']]

#BertzCT
train['Feature_10'] = [Descriptors.BertzCT(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_10'] = [Descriptors.BertzCT(mol) if mol is not None else None for mol in test['Feature_1']]

# Calculate the HeavyAtomMolWt descriptor
train['Feature_11'] = [Descriptors.HeavyAtomMolWt(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_11'] = [Descriptors.HeavyAtomMolWt(mol) if mol is not None else None for mol in test['Feature_1']]

# Calculate the number of hydrogen donors
train['Feature_12'] = [Lipinski.NumHDonors(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_12'] = [Lipinski.NumHDonors(mol) if mol is not None else None for mol in test['Feature_1']]

# Calculate the number of NumHAcceptors
train['Feature_13'] = [Descriptors.NumHAcceptors(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_13'] = [Descriptors.NumHAcceptors(mol) if mol is not None else None for mol in test['Feature_1']]

## calculate the Kappa1 descriptor
train['Feature_14'] = [rdMolDescriptors.CalcKappa1(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_14'] = [rdMolDescriptors.CalcKappa1(mol) if mol is not None else None for mol in test['Feature_1']]

#calculates the surface area of a molecule using the Labute ASA
train['Feature_15'] = [Descriptors.LabuteASA(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_15'] = [Descriptors.LabuteASA(mol) if mol is not None else None for mol in test['Feature_1']]

#calculates the number of rotatable bonds in a molecule
train['Feature_16'] = [Descriptors.NumRotatableBonds(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_16'] = [Descriptors.NumRotatableBonds(mol) if mol is not None else None for mol in test['Feature_1']]

#calculates the number of Hydrogen Bond Donors (HBD) in a molecule
train['Feature_17'] = [rdMolDescriptors.CalcNumHBD(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_17'] = [rdMolDescriptors.CalcNumHBD(mol) if mol is not None else None for mol in test['Feature_1']]

#calculates the number of Hydrogen Bond Acceptors (HBA) in a molecule
train['Feature_18'] = [rdMolDescriptors.CalcNumHBA(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_18'] = [rdMolDescriptors.CalcNumHBA(mol) if mol is not None else None for mol in test['Feature_1']]

# Get the number of rings in the molecule
train['Feature_19'] = [ Chem.rdMolDescriptors.CalcNumRings(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_19'] = [ Chem.rdMolDescriptors.CalcNumRings(mol) if mol is not None else None for mol in test['Feature_1']]

# Get the number of Aromatic Rings in the molecule
train['Feature_20'] = [ rdMolDescriptors.CalcNumAromaticRings(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_20'] = [ rdMolDescriptors.CalcNumAromaticRings(mol) if mol is not None else None for mol in test['Feature_1']]

# Calculate the Exact molecular weight of the molecule
train['Feature_21'] = [Chem.Descriptors.ExactMolWt(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_21'] = [Chem.Descriptors.ExactMolWt(mol) if mol is not None else None for mol in test['Feature_1']]

#the logarithm of the partition coefficient (logP),of a given molecule represented as a Mol object.
train['Feature_22'] = [Chem.Descriptors.MolLogP(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_22'] = [Chem.Descriptors.MolLogP(mol) if mol is not None else None for mol in test['Feature_1']]

# The mol.GetAtoms() function returns a list of all atoms in the molecule
train['Feature_23'] = [min([atom.GetDegree() for atom in mol.GetAtoms()]) if mol is not None else None for mol in train['Feature_1']]
test['Feature_23'] = [min([atom.GetDegree() for atom in mol.GetAtoms()]) if mol is not None else None for mol in test['Feature_1']]

# calculate the number of aromatic carbocycles in a given molecule
train['Feature_24'] = [Descriptors.NumAromaticCarbocycles(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_24'] = [Descriptors.NumAromaticCarbocycles(mol) if mol is not None else None for mol in test['Feature_1']]

# calculates the "Small Molecular Radius (SMR) Van der Waals Surface Area (VSA)" 
train['Feature_25'] = [Descriptors.SMR_VSA6(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_25'] = [Descriptors.SMR_VSA6(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the zero-order valence molecular connectivity index.
train['Feature_26'] = [Descriptors.Chi0v(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_26'] = [Descriptors.Chi0v(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the first-order valence molecular connectivity index.
train['Feature_27'] = [Descriptors.Chi1v(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_27'] = [Descriptors.Chi1v(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the second-order valence molecular connectivity index.
train['Feature_28'] = [Descriptors.Chi2v(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_28'] = [Descriptors.Chi2v(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the third-order valence molecular connectivity index.
train['Feature_29'] = [Descriptors.Chi3v(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_29'] = [Descriptors.Chi3v(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the fourth-order valence molecular connectivity index.
train['Feature_30'] = [Descriptors.Chi4v(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_30'] = [Descriptors.Chi4v(mol) if mol is not None else None for mol in test['Feature_1']]

#counts the number of benzene rings in a molecule.
train['Feature_31'] = [Descriptors.fr_benzene(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_31'] = [Descriptors.fr_benzene(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the number of aromatic rings
train['Feature_32'] = [Descriptors.NumAromaticRings(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_32'] = [Descriptors.NumAromaticRings(mol) if mol is not None else None for mol in test['Feature_1']]

#calculates the E-state contribution for a specific range of surface areas around a molecule
train['Feature_33'] = [Descriptors.VSA_EState6(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_33'] = [Descriptors.VSA_EState6(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the second-order molecular connectivity index, or chi-2n, for a molecule
train['Feature_34'] = [Descriptors.Chi2n(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_34'] = [Descriptors.Chi2n(mol) if mol is not None else None for mol in test['Feature_1']]

#calculate the third-order molecular connectivity index, or chi-3n, for a molecule
train['Feature_35'] = [Descriptors.Chi3n(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_35'] = [Descriptors.Chi3n(mol) if mol is not None else None for mol in test['Feature_1']]

#calculated based on the distance matrix of the molecule, Balaban J index for a molecule
train['Feature_36'] = [Descriptors.BalabanJ(mol) if mol is not None else None for mol in train['Feature_1']]
test['Feature_36'] = [Descriptors.BalabanJ(mol) if mol is not None else None for mol in test['Feature_1']]


In [34]:
train

,Id,Expected,Chemicals,AssayID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,...,Feature_27,Feature_28,Feature_29,Feature_30,Feature_31,Feature_32,Feature_33,Feature_34,Feature_35,Feature_36
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644,<rdkit.Chem.rdchem.Mol object at 0x7fd09100dc80>,317.599,19,2,78.3466,100,...,6.656814,6.526629,3.434823,2.659887,2,2,12.975791,4.157690,2.780170,2.474117
1,CCCCCCCCC(=O)C;2451,2,CCCCCCCCC(=O)C,2451,<rdkit.Chem.rdchem.Mol object at 0x7fd09100dba0>,156.269,11,7,48.6740,66,...,4.764784,3.219782,1.882392,1.154276,0,0,0.000000,3.219782,1.882392,2.849761
2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-];1384,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384,<rdkit.Chem.rdchem.Mol object at 0x7fd09100ddd0>,362.086,24,18,107.0624,148,...,10.941096,8.332693,5.233990,3.459239,0,0,0.000000,8.332693,5.233990,0.000000
3,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl;16,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16,<rdkit.Chem.rdchem.Mol object at 0x7fd09100dcf0>,255.665,17,3,62.0891,90,...,5.336317,3.872216,2.507966,1.656321,0,1,3.516012,3.484967,2.284387,2.011084
4,[Na+].[I-];1856,2,[Na+].[I-],1856,<rdkit.Chem.rdchem.Mol object at 0x7fd09100deb0>,149.894,2,0,0.0000,8,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75378,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N;33,2,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N,33,<rdkit.Chem.rdchem.Mol object at 0x7fd10303c510>,230.245,15,2,53.5809,82,...,5.401284,4.348518,2.921657,1.772269,1,1,5.429462,2.666116,1.593494,2.963576
75379,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C;1632,1,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C,1632,<rdkit.Chem.rdchem.Mol object at 0x7fd10303c580>,313.747,18,7,73.5820,104,...,8.556297,6.848877,4.476769,3.386806,0,1,0.165116,3.575776,1.827249,2.875062
75380,C1=CC=C2C(=C1)NC(=S)S2;1373,1,C1=CC=C2C(=C1)NC(=S)S2,1373,<rdkit.Chem.rdchem.Mol object at 0x7fd10303c5f0>,167.258,10,0,47.0097,50,...,4.164468,3.729544,2.541875,2.009906,1,2,8.106852,2.010679,1.326446,3.007620
75381,CCCCC(CC)C=O;2,2,CCCCC(CC)C=O,2,<rdkit.Chem.rdchem.Mol object at 0x7fd10303c660>,128.215,9,5,39.3700,54,...,3.799746,2.446639,1.645296,0.834703,0,0,0.000000,2.446639,1.645296,3.195412


In [35]:
train = train.dropna()


In [36]:
test = test.dropna()


In [37]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75377 entries, 0 to 75382
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id          75377 non-null  object 
 1   Expected    75377 non-null  int64  
 2   Chemicals   75377 non-null  object 
 3   AssayID     75377 non-null  object 
 4   Feature_1   75377 non-null  object 
 5   Feature_2   75377 non-null  float64
 6   Feature_3   75377 non-null  int64  
 7   Feature_4   75377 non-null  int64  
 8   Feature_5   75377 non-null  float64
 9   Feature_6   75377 non-null  int64  
 10  Feature_8   75377 non-null  float64
 11  Feature_9   75377 non-null  float64
 12  Feature_10  75377 non-null  float64
 13  Feature_11  75377 non-null  float64
 14  Feature_12  75377 non-null  int64  
 15  Feature_13  75377 non-null  int64  
 16  Feature_14  75377 non-null  float64
 17  Feature_15  75377 non-null  float64
 18  Feature_16  75377 non-null  int64  
 19  Feature_17  75377 non-nul

In [38]:
#Converting string to Int data type
train['AssayID'] = train['AssayID'].astype(int)
test['AssayID'] = test['AssayID'].astype(int)

In [39]:
#This code performs a binary classification task using the CatBoost algorithm to predict whether a molecule is toxic 
#or not based on a set of 36 input features. First, it subsets the input features and target variable from the 
#training data, and then splits the data into training and testing sets using a 80/20 ratio. Next, 
#it creates an instance of the CatBoostClassifier model with specific hyperparameters, and fits the model to the 
#entire training set. It then uses the trained model to predict the target variable for the test set, 
#and writes the predictions to a CSV file. Finally, it calculates the accuracy and F1 score of the 
#model using the predicted values and actual values from the test set, and prints the results to the console.

X_toxic = train[['AssayID','Feature_2','Feature_3','Feature_4','Feature_5','Feature_6','Feature_8','Feature_9'
                 ,'Feature_10','Feature_11','Feature_12','Feature_13','Feature_14','Feature_15','Feature_16'
                 ,'Feature_17','Feature_18','Feature_19','Feature_20','Feature_21','Feature_22','Feature_23'
                 ,'Feature_24','Feature_25','Feature_26','Feature_27','Feature_28','Feature_29','Feature_30'
                 ,'Feature_31','Feature_32','Feature_33','Feature_34','Feature_35','Feature_36']]
y_toxic= train['Expected']
#Passing Train and Test data 
X_train, X_test, y_train, y_test = train_test_split(X_toxic,y_toxic, test_size=0.2, random_state=10)

xg = CatBoostClassifier(iterations=25000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='Logloss')

#xg.fit(X_toxic, y_toxic)
xg.fit(X_train, y_train)

y_pred=xg.predict(X_test)
# predicted values from xgclassifier model
test_Input = test[['AssayID','Feature_2','Feature_3','Feature_4','Feature_5','Feature_6','Feature_8','Feature_9'
                 ,'Feature_10','Feature_11','Feature_12','Feature_13','Feature_14','Feature_15','Feature_16'
                 ,'Feature_17','Feature_18','Feature_19','Feature_20','Feature_21','Feature_22','Feature_23'
                 ,'Feature_24','Feature_25','Feature_26','Feature_27','Feature_28','Feature_29','Feature_30'
                 ,'Feature_31','Feature_32','Feature_33','Feature_34','Feature_35','Feature_36']]
xg_Predictions = xg.predict(test_Input)
xg_Data = pd.read_csv('test_II.csv')
xg_Data.reset_index(drop=True, inplace=True)

xg_Data['Predicted'] = xg_Predictions
xg_Data['Id']=xg_Data['x']
xg_Data=xg_Data[['Id','Predicted']]
xg_Data.to_csv('toxicity_Output.csv',index=False)

# Calculating Accuracy and f1 score
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1 Score:",f1_score(y_test, y_pred))

0:	learn: 0.6211342	total: 19.8ms	remaining: 8m 15s
1:	learn: 0.5643112	total: 28.8ms	remaining: 5m 59s
2:	learn: 0.5257620	total: 37.5ms	remaining: 5m 12s
3:	learn: 0.4855301	total: 46.2ms	remaining: 4m 48s
4:	learn: 0.4566607	total: 54.3ms	remaining: 4m 31s
5:	learn: 0.4373565	total: 62.7ms	remaining: 4m 21s
6:	learn: 0.4171637	total: 70.7ms	remaining: 4m 12s
7:	learn: 0.4071784	total: 79.5ms	remaining: 4m 8s
8:	learn: 0.3959991	total: 88ms	remaining: 4m 4s
9:	learn: 0.3848309	total: 97ms	remaining: 4m 2s
10:	learn: 0.3781801	total: 106ms	remaining: 3m 59s
11:	learn: 0.3715594	total: 115ms	remaining: 3m 59s
12:	learn: 0.3681568	total: 124ms	remaining: 3m 58s
13:	learn: 0.3640227	total: 133ms	remaining: 3m 56s
14:	learn: 0.3598209	total: 145ms	remaining: 4m 1s
15:	learn: 0.3558055	total: 153ms	remaining: 3m 59s
16:	learn: 0.3530261	total: 162ms	remaining: 3m 58s
17:	learn: 0.3497839	total: 171ms	remaining: 3m 57s
18:	learn: 0.3474908	total: 180ms	remaining: 3m 56s
19:	learn: 0.3460233

167:	learn: 0.2684097	total: 1.85s	remaining: 4m 32s
168:	learn: 0.2682220	total: 1.86s	remaining: 4m 33s
169:	learn: 0.2680258	total: 1.87s	remaining: 4m 32s
170:	learn: 0.2678693	total: 1.88s	remaining: 4m 33s
171:	learn: 0.2677320	total: 1.89s	remaining: 4m 33s
172:	learn: 0.2674568	total: 1.91s	remaining: 4m 34s
173:	learn: 0.2667091	total: 1.92s	remaining: 4m 34s
174:	learn: 0.2664484	total: 1.94s	remaining: 4m 34s
175:	learn: 0.2660733	total: 1.95s	remaining: 4m 35s
176:	learn: 0.2658166	total: 1.96s	remaining: 4m 35s
177:	learn: 0.2655317	total: 1.98s	remaining: 4m 35s
178:	learn: 0.2652630	total: 1.99s	remaining: 4m 35s
179:	learn: 0.2650424	total: 2s	remaining: 4m 35s
180:	learn: 0.2648662	total: 2.01s	remaining: 4m 35s
181:	learn: 0.2647132	total: 2.02s	remaining: 4m 35s
182:	learn: 0.2645233	total: 2.03s	remaining: 4m 35s
183:	learn: 0.2642795	total: 2.04s	remaining: 4m 35s
184:	learn: 0.2640654	total: 2.05s	remaining: 4m 35s
185:	learn: 0.2639058	total: 2.06s	remaining: 4m 

329:	learn: 0.2371347	total: 3.67s	remaining: 4m 34s
330:	learn: 0.2367657	total: 3.68s	remaining: 4m 34s
331:	learn: 0.2366699	total: 3.69s	remaining: 4m 34s
332:	learn: 0.2365307	total: 3.7s	remaining: 4m 34s
333:	learn: 0.2363583	total: 3.71s	remaining: 4m 34s
334:	learn: 0.2361388	total: 3.73s	remaining: 4m 34s
335:	learn: 0.2360535	total: 3.74s	remaining: 4m 34s
336:	learn: 0.2359212	total: 3.75s	remaining: 4m 34s
337:	learn: 0.2358292	total: 3.76s	remaining: 4m 34s
338:	learn: 0.2356687	total: 3.77s	remaining: 4m 34s
339:	learn: 0.2354920	total: 3.78s	remaining: 4m 34s
340:	learn: 0.2353540	total: 3.79s	remaining: 4m 34s
341:	learn: 0.2351913	total: 3.8s	remaining: 4m 34s
342:	learn: 0.2350827	total: 3.81s	remaining: 4m 34s
343:	learn: 0.2348519	total: 3.82s	remaining: 4m 34s
344:	learn: 0.2346391	total: 3.84s	remaining: 4m 34s
345:	learn: 0.2345536	total: 3.85s	remaining: 4m 34s
346:	learn: 0.2343943	total: 3.86s	remaining: 4m 34s
347:	learn: 0.2342549	total: 3.87s	remaining: 4m

491:	learn: 0.2173530	total: 5.48s	remaining: 4m 33s
492:	learn: 0.2172537	total: 5.49s	remaining: 4m 33s
493:	learn: 0.2171705	total: 5.5s	remaining: 4m 33s
494:	learn: 0.2171247	total: 5.51s	remaining: 4m 33s
495:	learn: 0.2170090	total: 5.53s	remaining: 4m 33s
496:	learn: 0.2168682	total: 5.54s	remaining: 4m 33s
497:	learn: 0.2168275	total: 5.55s	remaining: 4m 33s
498:	learn: 0.2167414	total: 5.56s	remaining: 4m 33s
499:	learn: 0.2166480	total: 5.57s	remaining: 4m 32s
500:	learn: 0.2165482	total: 5.58s	remaining: 4m 32s
501:	learn: 0.2164580	total: 5.59s	remaining: 4m 32s
502:	learn: 0.2163351	total: 5.6s	remaining: 4m 32s
503:	learn: 0.2162795	total: 5.61s	remaining: 4m 32s
504:	learn: 0.2162105	total: 5.62s	remaining: 4m 32s
505:	learn: 0.2160689	total: 5.63s	remaining: 4m 32s
506:	learn: 0.2159526	total: 5.65s	remaining: 4m 32s
507:	learn: 0.2158396	total: 5.66s	remaining: 4m 32s
508:	learn: 0.2157834	total: 5.67s	remaining: 4m 32s
509:	learn: 0.2156259	total: 5.68s	remaining: 4m

655:	learn: 0.2030044	total: 7.29s	remaining: 4m 30s
656:	learn: 0.2029364	total: 7.3s	remaining: 4m 30s
657:	learn: 0.2028761	total: 7.32s	remaining: 4m 30s
658:	learn: 0.2028186	total: 7.33s	remaining: 4m 30s
659:	learn: 0.2027631	total: 7.34s	remaining: 4m 30s
660:	learn: 0.2026790	total: 7.35s	remaining: 4m 30s
661:	learn: 0.2026264	total: 7.36s	remaining: 4m 30s
662:	learn: 0.2025825	total: 7.37s	remaining: 4m 30s
663:	learn: 0.2025060	total: 7.38s	remaining: 4m 30s
664:	learn: 0.2024122	total: 7.39s	remaining: 4m 30s
665:	learn: 0.2022248	total: 7.4s	remaining: 4m 30s
666:	learn: 0.2021588	total: 7.41s	remaining: 4m 30s
667:	learn: 0.2020988	total: 7.42s	remaining: 4m 30s
668:	learn: 0.2020263	total: 7.43s	remaining: 4m 30s
669:	learn: 0.2019780	total: 7.45s	remaining: 4m 30s
670:	learn: 0.2019014	total: 7.46s	remaining: 4m 30s
671:	learn: 0.2018381	total: 7.47s	remaining: 4m 30s
672:	learn: 0.2017865	total: 7.48s	remaining: 4m 30s
673:	learn: 0.2017051	total: 7.49s	remaining: 4m

...

24357:	learn: 0.0285720	total: 4m 11s	remaining: 6.62s
24358:	learn: 0.0285703	total: 4m 11s	remaining: 6.61s
24359:	learn: 0.0285663	total: 4m 11s	remaining: 6.6s
24360:	learn: 0.0285663	total: 4m 11s	remaining: 6.59s
24361:	learn: 0.0285662	total: 4m 11s	remaining: 6.58s
24362:	learn: 0.0285640	total: 4m 11s	remaining: 6.57s
24363:	learn: 0.0285638	total: 4m 11s	remaining: 6.56s
24364:	learn: 0.0285638	total: 4m 11s	remaining: 6.55s
24365:	learn: 0.0285608	total: 4m 11s	remaining: 6.54s
24366:	learn: 0.0285592	total: 4m 11s	remaining: 6.53s
24367:	learn: 0.0285592	total: 4m 11s	remaining: 6.52s
24368:	learn: 0.0285592	total: 4m 11s	remaining: 6.5s
24369:	learn: 0.0285592	total: 4m 11s	remaining: 6.5s
24370:	learn: 0.0285592	total: 4m 11s	remaining: 6.48s
24371:	learn: 0.0285589	total: 4m 11s	remaining: 6.47s
24372:	learn: 0.0285589	total: 4m 11s	remaining: 6.46s
24373:	learn: 0.0285589	total: 4m 11s	remaining: 6.45s
24374:	learn: 0.0285589	total: 4m 11s	remaining: 6.44s
24375:	learn:

24516:	learn: 0.0285347	total: 4m 13s	remaining: 4.99s
24517:	learn: 0.0285343	total: 4m 13s	remaining: 4.98s
24518:	learn: 0.0285343	total: 4m 13s	remaining: 4.97s
24519:	learn: 0.0285343	total: 4m 13s	remaining: 4.96s
24520:	learn: 0.0285336	total: 4m 13s	remaining: 4.95s
24521:	learn: 0.0285334	total: 4m 13s	remaining: 4.94s
24522:	learn: 0.0285334	total: 4m 13s	remaining: 4.93s
24523:	learn: 0.0285330	total: 4m 13s	remaining: 4.92s
24524:	learn: 0.0285330	total: 4m 13s	remaining: 4.91s
24525:	learn: 0.0285329	total: 4m 13s	remaining: 4.9s
24526:	learn: 0.0285296	total: 4m 13s	remaining: 4.89s
24527:	learn: 0.0285290	total: 4m 13s	remaining: 4.88s
24528:	learn: 0.0285286	total: 4m 13s	remaining: 4.87s
24529:	learn: 0.0285286	total: 4m 13s	remaining: 4.86s
24530:	learn: 0.0285283	total: 4m 13s	remaining: 4.85s
24531:	learn: 0.0285283	total: 4m 13s	remaining: 4.84s
24532:	learn: 0.0285283	total: 4m 13s	remaining: 4.83s
24533:	learn: 0.0285282	total: 4m 13s	remaining: 4.82s
24534:	lear

24667:	learn: 0.0284452	total: 4m 15s	remaining: 3.44s
24668:	learn: 0.0284452	total: 4m 15s	remaining: 3.43s
24669:	learn: 0.0284452	total: 4m 15s	remaining: 3.42s
24670:	learn: 0.0284451	total: 4m 15s	remaining: 3.41s
24671:	learn: 0.0284451	total: 4m 15s	remaining: 3.4s
24672:	learn: 0.0284451	total: 4m 15s	remaining: 3.38s
24673:	learn: 0.0284451	total: 4m 15s	remaining: 3.38s
24674:	learn: 0.0284451	total: 4m 15s	remaining: 3.36s
24675:	learn: 0.0284450	total: 4m 15s	remaining: 3.35s
24676:	learn: 0.0284450	total: 4m 15s	remaining: 3.34s
24677:	learn: 0.0284428	total: 4m 15s	remaining: 3.33s
24678:	learn: 0.0284428	total: 4m 15s	remaining: 3.32s
24679:	learn: 0.0284427	total: 4m 15s	remaining: 3.31s
24680:	learn: 0.0284427	total: 4m 15s	remaining: 3.3s
24681:	learn: 0.0284427	total: 4m 15s	remaining: 3.29s
24682:	learn: 0.0284426	total: 4m 15s	remaining: 3.28s
24683:	learn: 0.0284426	total: 4m 15s	remaining: 3.27s
24684:	learn: 0.0284423	total: 4m 15s	remaining: 3.26s
24685:	learn

24843:	learn: 0.0283899	total: 4m 17s	remaining: 1.62s
24844:	learn: 0.0283899	total: 4m 17s	remaining: 1.61s
24845:	learn: 0.0283899	total: 4m 17s	remaining: 1.59s
24846:	learn: 0.0283899	total: 4m 17s	remaining: 1.58s
24847:	learn: 0.0283899	total: 4m 17s	remaining: 1.57s
24848:	learn: 0.0283899	total: 4m 17s	remaining: 1.56s
24849:	learn: 0.0283835	total: 4m 17s	remaining: 1.55s
24850:	learn: 0.0283792	total: 4m 17s	remaining: 1.54s
24851:	learn: 0.0283766	total: 4m 17s	remaining: 1.53s
24852:	learn: 0.0283766	total: 4m 17s	remaining: 1.52s
24853:	learn: 0.0283695	total: 4m 17s	remaining: 1.51s
24854:	learn: 0.0283695	total: 4m 17s	remaining: 1.5s
24855:	learn: 0.0283694	total: 4m 17s	remaining: 1.49s
24856:	learn: 0.0283694	total: 4m 17s	remaining: 1.48s
24857:	learn: 0.0283694	total: 4m 17s	remaining: 1.47s
24858:	learn: 0.0283669	total: 4m 17s	remaining: 1.46s
24859:	learn: 0.0283669	total: 4m 17s	remaining: 1.45s
24860:	learn: 0.0283669	total: 4m 17s	remaining: 1.44s
24861:	lear

24996:	learn: 0.0282967	total: 4m 18s	remaining: 31ms
24997:	learn: 0.0282967	total: 4m 18s	remaining: 20.7ms
24998:	learn: 0.0282967	total: 4m 18s	remaining: 10.3ms
24999:	learn: 0.0282967	total: 4m 18s	remaining: 0us
Accuracy: 0.9076014858052533
F1 Score: 0.6488530375598689
